# Stacked Autoencoder for MNIST Dataset

### 패키지

In [1]:
import numpy as np
import math

import tensorflow as tf

import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical

from sklearn.manifold import TSNE
from skimage import io

import theano

import json

Using TensorFlow backend.


In [2]:
 # Functions 
def get_hot_idx(arr):
    return arr.index(max(arr))

def save_to_json_file(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)
    print(filename + ' 저장완료')
    
def get_round_array(array, decimal):
    return [round(e, decimal) for e in array]

def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
    activations = get_activations([X_batch,0])
    return activations

def get_arr_from_json_file(filename):
    input_file = open (filename)
    return json.load(input_file)
    
current_milli_time = lambda: int(round(time.time() * 1000))

### 데이터셋 로드

In [3]:
image_width = 28
image_height = 28
num_of_features = image_width * image_height

num_of_class = 10
num_of_trainset = 60000
num_of_testset = 10000

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(num_of_trainset, num_of_features).astype('float32') / 255.0
X_test = X_test.reshape(num_of_testset, num_of_features).astype('float32') / 255.0
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

In [4]:
# 로컬 테스트 데이터 로드
images = np.zeros((10000, 784))
for real in range(10):
    for idx in range(1, 1001):
        file = '../../data/mnist/images/'+ str(real) + '/' + str(real) + '_' + str(idx) +'.png'
        image = np.ndarray.flatten(io.imread(file)) / 255.0
        image = np.array([1 - pixel for pixel in image])
        images[real * 1000 + idx - 1] = image

### 오토인코더 모델 구성

In [5]:
# 모델 구축 
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

W0716 18:52:09.098741 4613531072 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0716 18:52:09.111839 4613531072 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0716 18:52:09.115370 4613531072 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0716 18:52:09.182354 4613531072 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0716 18:52:09.195177 4613531072 deprecation_wrapper.py:119] From /anaconda3

### 오토인코더 모델 학습

In [6]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

W0716 18:52:09.407172 4613531072 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 44us/step - loss: 0.3260 - val_loss: 0.2631
Epoch 2/50
60000/60000 [==============================] - 2s 41us/step - loss: 0.2544 - val_loss: 0.2433
Epoch 3/50
60000/60000 [==============================] - 2s 41us/step - loss: 0.2325 - val_loss: 0.2225
Epoch 4/50
60000/60000 [==============================] - 2s 41us/step - loss: 0.2173 - val_loss: 0.2096
Epoch 5/50
60000/60000 [==============================] - 3s 42us/step - loss: 0.2045 - val_loss: 0.1971
Epoch 6/50
60000/60000 [==============================] - 3s 42us/step - loss: 0.1907 - val_loss: 0.1830
Epoch 7/50
60000/60000 [==============================] - 2s 39us/step - loss: 0.1813 - val_loss: 0.1773
Epoch 8/50
60000/60000 [==============================] - 2s 39us/step - loss: 0.1742 - val_loss: 0.1695
Epoch 9/50
60000/60000 [==============================] - 2s 39us/step - loss: 0.1675 - val_loss: 0.1637
Epoch

### image reconstruction with autoencoder

In [7]:
X_train_decoded = autoencoder.predict(X_train)
X_test_decoded = autoencoder.predict(X_test)

### 예측 모델

In [8]:
# 모델 구축 
model = Sequential()
model.add(Dense(units=16, input_dim=num_of_features, activation='relu'))
model.add(Dense(units=num_of_class, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [17]:
# 학습
model.fit(X_train_decoded, Y_train, epochs=10, batch_size=32)

Epoch 1/10
60000/60000 [==============================] - 1s 21us/step - loss: 0.2837 - acc: 0.9152
Epoch 2/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.2736 - acc: 0.9184
Epoch 3/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.2646 - acc: 0.9211
Epoch 4/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.2567 - acc: 0.9234
Epoch 5/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.2499 - acc: 0.9255
Epoch 6/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.2429 - acc: 0.9280
Epoch 7/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.2373 - acc: 0.9301
Epoch 8/10
60000/60000 [==============================] - 1s 21us/step - loss: 0.2326 - acc: 0.9311
Epoch 9/10
60000/60000 [==============================] - 1s 21us/step - loss: 0.2275 - acc: 0.9325
Epoch 10/10
60000/60000 [==============================] - 1s 24us/step - loss: 0.2235 - acc: 0.9334

### 테스트 셋 결과 확인

In [18]:
loss_and_metrics = model.evaluate(X_test_decoded, Y_test, batch_size=32)
print('loss_and_metrics : ' + str(loss_and_metrics))

10000/10000 [==============================] - 0s 11us/step
loss_and_metrics : [0.21835300922989845, 0.9369]


## 내 데이터로 테스트

In [24]:
images_decoded = autoencoder.predict(images)

In [25]:
# 예측
pred_proba =  model.predict_proba(images_decoded).tolist()

In [26]:
# 성능 확인 및 저장
correct = 0
performances = {
    "accuracy": 0,
    "recall": [0] * 10,
    "precision": [0] * 10
}
truePredict = [0] * 10
numOfPredict = [0] * 10
predicts = []

for i in range(len(pred_proba)):
    prob = pred_proba[i]
    pred = get_hot_idx(prob)
    real = i // 1000
    predicts.append(dict({
        "real": real,
        "pred": pred,
        "prob": [round(e, 4) for e in prob]
    }))
    numOfPredict[pred] = numOfPredict[pred] + 1
    if pred is real:
        truePredict[real] = truePredict[real] + 1
        correct = correct + 1

performances["accuracy"] = correct / 10000
performances["recall"] = [round(truePredict[i] / 1000, 4) for i in range(10) ]
performances["precision"] = [round(truePredict[i] / numOfPredict[i], 4) for i in range(10)]

print(performances)

{'accuracy': 0.9283, 'recall': [0.964, 0.956, 0.927, 0.926, 0.939, 0.915, 0.949, 0.931, 0.884, 0.892], 'precision': [0.9507, 0.9696, 0.9345, 0.9223, 0.9055, 0.9123, 0.9295, 0.9319, 0.9095, 0.9177]}


### 차원축소

1) 마지막 직전의 레이어에서 activation values를 뽑는다.

2) t-SNE로 차원을 축소한다.

In [27]:
samples = []
idxs = get_arr_from_json_file('./sample_image_idxs.json')
for idx in idxs:
    samples.append(images[idx])
print(len(samples))

500


In [28]:
extracted_features = get_activations(model, -2, samples)[0]
y = TSNE(n_components=2).fit_transform(extracted_features)
sne_map = []
for e in y:
    sne_map.append({
        "x": round(e[0], 2),
        "y": round(e[1], 2)
    })

In [29]:
model_data = {
    "model_name": 'Single Layer Perceptron with Stacked Autoencder',
    "short_name": 'SAE',
    "description": '오토인코더 모델을 통해 정제한 데이터를 통해 퍼셉트론으로 예측하였다.',
    "performance": {
        "accuracy": performances["accuracy"],
        "recall": performances["recall"],
        "precision": performances["precision"]
    },
    "predict": predicts,
    "t-sne": sne_map
}

In [30]:
with open('mnist_sae.json', 'w', encoding='utf-8') as f:
    json.dump(str(model_data), f, ensure_ascii=False, indent=4)